<a href="https://colab.research.google.com/github/santoshrsarangi/tensorflow/blob/main/computer-vision/2_Cats_vs_Dogs_Kaggle_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>